In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)
from mmcv import Config
import mmcv
from mmdet.apis import set_random_seed
# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
from mmcv.runner import load_checkpoint
from mmdet.apis import multi_gpu_test, single_gpu_test
print(get_compiling_cuda_version())
print(get_compiler_version())
from mmdet.datasets import build_dataset, build_dataloader
from mmdet.models import build_detector
from mmdet.apis import train_detector
import copy
import os.path as osp
from mmcv.parallel import MMDataParallel
import mmcv
import numpy as np
import json
import os
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
import matplotlib.pyplot as plt
from pycocotools.coco import COCO

## Visualize the dataset

In [ ]:
cfg = Config.fromfile('configs/spine/mask_rcnn_r50_fpn_4x_coco.py')
datasets = [build_dataset(cfg.data.train)]
datasets

## Train the netwrok

In [ ]:
cfg = Config.fromfile('configs/spine/mask_rcnn_r50_fpn_4x_coco.py')
# Set up working dir to save files and logs.
cfg.work_dir = 'work_dirs/mask_rcnn_r50_fpn_2x_coco_cp4'
seed = 1
# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.001
#cfg.lr_config.warmup = 'linear'
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
#cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 24

# Set seed thus the results are more reproducible
cfg.seed = seed
set_random_seed(seed, deterministic=False)
cfg.gpu_ids = [0]
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.init_weights()
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

## Evaluate the result

In [ ]:
cfg = Config.fromfile('configs/spine/mask_rcnn_r50_fpn_4x_coco.py')
# Set up working dir to save files and logs.
cfg.work_dir = 'work_dirs/mask_rcnn_r50_fpn_2x_coco_cp4'
dataset = build_dataset(cfg.data.test)
checkpoint = cfg.work_dir +'/latest.pth'
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.val.pop('samples_per_gpu', 1),
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')
model.CLASSES = checkpoint['meta']['CLASSES']
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)
metric = dataset.evaluate(outputs, metric=['bbox', 'segm'],classwise=True)

## calculate dice for all cases

In [ ]:
def dice(im1, im2):
    """
    dice for images
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("im1 and im2 must have the same shape.")

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / (im1.sum() + im2.sum())

In [ ]:
label = COCO('raw_val_label.json')
img_ids = label.getImgIds()
dice_list = []
for i in (img_ids):
    img = label.imgs[i]
    anns_id = label.getAnnIds(imgIds=img['id'])
    file_name = img['file_name']
    img = mmcv.imread('detection_images/val/' +file_name)
    model.cfg = cfg
    result = inference_detector(model, img)
    anns = label.loadAnns(anns_id)
    mask = label.annToMask(anns[0])
    result_mask = result[1][0][0]
    dice_list.append(dice(mask,result_mask))

In [ ]:
np.array(dice_list).mean()